In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

In [30]:
from keras.layers import LeakyReLU

In [2]:
# pip install keras-tuner --upgrade

     -------------------------------------- 129.1/129.1 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
from kerastuner.tuners import RandomSearch

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_25604\2610907151.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [4]:
df = pd.read_csv(r"C:\Users\Lenovo\ds\datasets\Real_Combine.csv")

In [5]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [6]:
X = df.iloc[:,:-1]
X

,T,TM,Tm,SLP,H,VV,V,VM
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2
...,...,...,...,...,...,...,...,...
1088,18.1,24.0,11.2,1015.4,56.0,1.8,15.9,25.9
1089,17.8,25.0,10.7,1015.8,54.0,2.3,9.4,22.2
1090,13.9,24.5,11.4,1015.0,95.0,0.6,8.7,14.8
1091,16.3,23.0,9.8,1016.9,78.0,1.1,7.4,16.5


In [7]:
Y = df.iloc[:,-1]
Y

0       219.720833
1       182.187500
2       154.037500
3       223.208333
4       200.645833
           ...    
1088    288.416667
1089    256.833333
1090    169.000000
1091    186.041667
1092    185.583333
Name: PM 2.5, Length: 1093, dtype: float64

## Hyperparameters
### How many number of hidden layers we should have?
### How many number of neurons we should have in hidden layers?
### Learning Rate



In [37]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2,20)):
        model.add(layers.Dense(units=hp.Int('units_'+ str(i), min_value=32, max_value=512, step=32)))
        model.add(LeakyReLU(alpha=0.1))
        model.add(layers.Dense(1, activation='linear'))
    
    model.compile(
        optimizer = keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss = 'mse',
        metrics = ['mse']
    )
    
    return model

In [38]:
tuner = RandomSearch(
    build_model,
    objective = 'val_loss',
    max_trials = 5,
    executions_per_trial = 3,
    directory = 'models',
    project_name='Air Quality Index'
)

Reloading Tuner from models\Air Quality Index\tuner0.json


In [39]:
tuner.search_space_summary()

Search space summary
Default search space size: 15
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_6

In [40]:
from sklearn.model_selection import train_test_split


In [41]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

In [42]:
tuner.search(X_train, Y_train, 
            epochs=10, 
             validation_data = (X_test, Y_test))


Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
20                |13                |num_layers
256               |288               |units_0
384               |352               |units_1
0.001             |0.001             |learning_rate
32                |256               |units_2
128               |224               |units_3
128               |256               |units_4
288               |352               |units_5
192               |448               |units_6
480               |224               |units_7
512               |320               |units_8
320               |32                |units_9
256               |32                |units_10
256               |32                |units_11
96                |32                |units_12
32                |None              |units_13
32                |None              |units_14
32                |None              |units_15
32                |None              |units_16
32                |None       

C:\Users\Lenovo\anaconda3\lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 28s 65ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 10/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - 

RuntimeError: Number of consecutive failures exceeded the limit of 3.


In [ ]:
# yf-h2urb6